In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = 16, 12


from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
from sklearn.decomposition import PCA

from sklearn.learning_curve import learning_curve
from sklearn.learning_curve import validation_curve

##### The Sequential Backward Selection algorithm below has been taken from:
Python Machine Learning - By Sebastian Raschka - Chapter 4

In [ ]:
from sklearn.base import clone
from itertools import combinations
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

class SBS():
    def __init__(self, estimator, k_features, scoring=accuracy_score,
                 test_size=0.25, random_state=1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y):
        
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=self.test_size,
                             random_state=self.random_state)

        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train, 
                                 X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train, 
                                         X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]

        return self

    def transform(self, X):
        return X[:, self.indices_]

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])
        score = self.scoring(y_test, y_pred)
        return score

In [ ]:
file = 'data/price_data.csv'
data = pd.read_csv(file)

In [ ]:
data.head(3)

In [ ]:
del data['Date']

In [ ]:
data.head(3)

In [ ]:
# Features are F1 through F34
# Labels are RET5, RET10, RET15, RET20, RET25, RET30

X = data.loc[:,'F1':'F34']
y5 = data.loc[:,'RET5']
y10 = data.loc[:,'RET10']
y15 = data.loc[:,'RET15']
y20 = data.loc[:,'RET20']
y25 = data.loc[:,'RET25']
y30 = data.loc[:,'RET30']

In [ ]:
X.head(5)

In [ ]:
#y5.head(5)
#y10.head(5)
#y15.head(5)
#y20.head(5)
#y25.head(5)
y30.head(5)

In [ ]:
X.describe()

In [ ]:
y5.describe()

When we plot histograms for each return label we can see that all the returns are approximately normally distributed aroung 0. We can also see that this stock in particular has a positive skew. This conforms with the upward bias in the stock market.

In [ ]:
plt.rcParams['figure.figsize'] = 8, 8
y5.hist()
plt.xlabel('Return')
plt.ylabel('# of samples')
plt.title('5 Day Return')
plt.show()

In [ ]:
y10.hist()
plt.xlabel('Return')
plt.ylabel('# of samples')
plt.title('10 Day Return')
plt.show()

In [ ]:
y15.hist()
plt.xlabel('Return')
plt.ylabel('# of samples')
plt.title('15 Day Return')
plt.show()

In [ ]:
y20.hist()
plt.xlabel('Return')
plt.ylabel('# of samples')
plt.title('20 Day Return')
plt.show()

In [ ]:
y25.hist()
plt.xlabel('Return')
plt.ylabel('# of samples')
plt.title('25 Day Return')
plt.show()

In [ ]:
y30.hist()
plt.xlabel('Return')
plt.ylabel('# of samples')
plt.title('30 Day Return')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 15, 8
fig, axes = plt.subplots(2,3, sharex=False, sharey=False)

axes[0,0].hist(y5)
axes[0,0].set_title('5 Day Return', fontsize=15)

axes[0,1].hist(y10)
axes[0,1].set_title('10 Day Return', fontsize=15)

axes[0,2].hist(y15)
axes[0,2].set_title('15 Day Return', fontsize=15)

axes[1,0].hist(y20)
axes[1,0].set_title('20 Day Return', fontsize=15)

axes[1,1].hist(y25)
axes[1,1].set_title('25 Day Return', fontsize=15)

axes[1,2].hist(y30)
axes[1,2].set_title('30 Day Return', fontsize=15)

plt.subplots_adjust(wspace=0.5, hspace=0.5)

### Phase 1
In the first phase of model building we will start with the simplest label grouping. We will separate our labels into either positive, or negative returns. This will give us two classes with approximately equal class distributions.

In [ ]:
y5.columns = ['RET']
y10.columns = ['RET']
y15.columns = ['RET']
y20.columns = ['RET']
y25.columns = ['RET']
y30.columns = ['RET']

In [ ]:
def binary_class(row):
    if row < 0.00:
        return 0
    else:
        return 1

In [ ]:
y5_b = pd.Series(index=y5.index)
y10_b = pd.Series(index=y5.index)
y15_b = pd.Series(index=y5.index)
y20_b = pd.Series(index=y5.index)
y25_b = pd.Series(index=y5.index)
y30_b = pd.Series(index=y5.index)

y5_b = y5.apply(binary_class)
y10_b = y10.apply(binary_class)
y15_b = y15.apply(binary_class)
y20_b = y20.apply(binary_class)
y25_b = y25.apply(binary_class)
y30_b = y30.apply(binary_class)

In [ ]:
y25_b.head(5)

In [ ]:
plt.rcParams['figure.figsize'] = 8, 8

Looking at the binary classification results, we can see that separating the class labels based on negative and positive returns, gives us an approximately equal distribution for each class.

In [ ]:
y5_b.hist()
plt.xlabel('Negative or Positive Return')
plt.ylabel('# of samples')
plt.title('Binary Classification')
plt.show()

### Phase 2
In this phase we will increase the number of boundaries from one to two. From a trading standpoint, we want to stay away from trades that return close to 0% return. We do want to identify trades that have substantial return.

We can define substantial return as > 2% or 0.02 or < -2% or -0.02
Class boundaries:
label 0: <=-0.02
label 1: >-0.02 and < 0.02
label 2: >= 0.02

Another way to find out what the boundary should be is to look at the class distributions that result from a particular boundary. The boundary can then be changed to ensure that the distribution becomes approximately equal for all classes. You may find that you have to use different boundaries for different returns.

In [ ]:
def update_return_class(row, neg_cutoff, pos_cutoff):
    if row <= neg_cutoff:
        return 0
    elif row > neg_cutoff and row < pos_cutoff:
        return 1
    elif row >= pos_cutoff:
        return 2

In [ ]:
y5_t = pd.Series(index=y5.index)
y10_t = pd.Series(index=y5.index)
y15_t = pd.Series(index=y5.index)
y20_t = pd.Series(index=y5.index)
y25_t = pd.Series(index=y5.index)
y30_t = pd.Series(index=y5.index)

y5_t = y5.apply(update_return_class, args=(-0.02, 0.02))
y10_t = y10.apply(update_return_class, args=(-0.03, 0.03))
y15_t = y15.apply(update_return_class, args=(-0.05, 0.05))
y20_t = y20.apply(update_return_class, args=(-0.05, 0.05))
y25_t = y25.apply(update_return_class, args=(-0.05, 0.07))
y30_t = y30.apply(update_return_class, args=(-0.05, 0.08))

In [ ]:
y5_t.hist()
plt.xlabel('Trinary Classification Return')
plt.ylabel('# of samples')
plt.title('5 Day Return')
plt.show()

In [ ]:
y10_t.hist()
plt.xlabel('Trinary Classification Return')
plt.ylabel('# of samples')
plt.title('10 Day Return')
plt.show()

In [ ]:
y15_t.hist()
plt.xlabel('Trinary Classification Return')
plt.ylabel('# of samples')
plt.title('15 Day Return')
plt.show()

In [ ]:
y20_t.hist()
plt.xlabel('Trinary Classification Return')
plt.ylabel('# of samples')
plt.title('20 Day Return')
plt.show()

In [ ]:
y25_t.hist()
plt.xlabel('Trinary Classification Return')
plt.ylabel('# of samples')
plt.title('25 Day Return')
plt.show()

In [ ]:
y30_t.hist()
plt.xlabel('Trinary Classification Return')
plt.ylabel('# of samples')
plt.title('30 Day Return')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 15, 8
fig, axes = plt.subplots(2,3, sharex=False, sharey=False)

axes[0,0].hist(y5_t)
axes[0,0].set_title('5 Day Return', fontsize=15)

axes[0,1].hist(y10_t)
axes[0,1].set_title('10 Day Return', fontsize=15)

axes[0,2].hist(y15_t)
axes[0,2].set_title('15 Day Return', fontsize=15)

axes[1,0].hist(y20_t)
axes[1,0].set_title('20 Day Return', fontsize=15)

axes[1,1].hist(y25_t)
axes[1,1].set_title('25 Day Return', fontsize=15)

axes[1,2].hist(y30_t)
axes[1,2].set_title('30 Day Return', fontsize=15)

plt.subplots_adjust(wspace=0.5, hspace=0.5)

### Testing and Validation Strategy
We will be using multi-pronged testing strategy
1) Use k-fold cross-validation
2) Use continuous samples as training data and test on future data
3) Use sliding window continuous training and test data

In [ ]:
validation_size = 0.20
seed = 7
X5b_train, X5b_validation, Y5b_train, Y5b_validation = cross_validation.train_test_split(X, y5_b, test_size=validation_size, 
                                                                                 random_state=seed)
X10b_train, X10b_validation, Y10b_train, Y10b_validation = cross_validation.train_test_split(X, y10_b, test_size=validation_size, 
                                                                                 random_state=seed)
X15b_train, X15b_validation, Y15b_train, Y15b_validation = cross_validation.train_test_split(X, y15_b, test_size=validation_size, 
                                                                                 random_state=seed)
X20b_train, X20b_validation, Y20b_train, Y20b_validation = cross_validation.train_test_split(X, y20_b, test_size=validation_size, 
                                                                                 random_state=seed)
X25b_train, X25b_validation, Y25b_train, Y25b_validation = cross_validation.train_test_split(X, y25_b, test_size=validation_size, 
                                                                                 random_state=seed)
X30b_train, X30b_validation, Y30b_train, Y30b_validation = cross_validation.train_test_split(X, y30_b, test_size=validation_size, 
                                                                                 random_state=seed)

X5t_train, X5t_validation, Y5t_train, Y5t_validation = cross_validation.train_test_split(X, y5_t, test_size=validation_size, 
                                                                                 random_state=seed)
X10t_train, X10t_validation, Y10t_train, Y10t_validation = cross_validation.train_test_split(X, y10_t, test_size=validation_size, 
                                                                                 random_state=seed)
X15t_train, X15t_validation, Y15t_train, Y15t_validation = cross_validation.train_test_split(X, y15_t, test_size=validation_size, 
                                                                                 random_state=seed)
X20t_train, X20t_validation, Y20t_train, Y20t_validation = cross_validation.train_test_split(X, y20_t, test_size=validation_size, 
                                                                                 random_state=seed)
X25t_train, X25t_validation, Y25t_train, Y25t_validation = cross_validation.train_test_split(X, y25_t, test_size=validation_size, 
                                                                                 random_state=seed)
X30t_train, X30t_validation, Y30t_train, Y30t_validation = cross_validation.train_test_split(X, y30_t, test_size=validation_size, 
                                                                                 random_state=seed)

In [ ]:
# Evaluate Algorithms
# Test options and evaluation metric
num_folds = 10
num_instances = len(X10b_train)
seed = 7
scoring = 'accuracy'

In [ ]:
xgb = XGBClassifier()
xgb.fit(X10b_train, Y10b_train)
Y_pred = xgb.predict(X10b_validation)
plot_importance(xgb)


rf = RandomForestClassifier()
et = ExtraTreesClassifier()
xgb = XGBClassifier()
gbc = GradientBoostingClassifier()

stdsc = StandardScaler()
X10b_train_std = stdsc.fit_transform(X10b_train)
X10b_test_std = stdsc.transform(X10b_validation)

plt.show()

In [ ]:
#rfecv = RFECV(estimator=et, step=1, cv=StratifiedKFold(y10_b, n_folds=2, random_state=seed), scoring='accuracy')
rfecv = RFECV(estimator=et, step=1, scoring='accuracy')
#print(len(X10b_train_std))
#print(len(Y10b_train))

rfecv.fit(X10b_train_std,Y10b_train)
rfe = RFE(estimator=et, step=1)
rfe.fit(X10b_train_std,Y10b_train)

In [ ]:
print('# of selected features with CV: %i' %rfecv.n_features_)
print('Mask of selected features with CV: %s' %rfecv.support_)
print('Selected features ranking: %s' %rfecv.ranking_)
print('CV scores: %s' %rfecv.grid_scores_)

In [ ]:
print('Original number of features is %s' % X.shape[1])
print("RFE final number of features : %d" % rfe.n_features_)
print("RFECV final number of features : %d" % rfecv.n_features_)
print('')

import numpy as np
g_scores = rfecv.grid_scores_
indices = np.argsort(g_scores)[::-1]
print('Printing RFECV results:')
for f in range(X.shape[1]):
    print("%d. Number of features: %d; Grid_Score: %f" % (f + 1, indices[f]+1, g_scores[indices[f]]))

In [ ]:
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
print(rfecv.ranking_)

In [ ]:
print(rfecv.support_)

In [ ]:
print(rfecv.n_features_)

In [ ]:
for selected, feature in zip(rfecv.support_, X.columns):
    if selected == True:
        print(feature)

In [ ]:
# ensembles
ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB', AdaBoostClassifier())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingClassifier())])))
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestClassifier())])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesClassifier())])))
ensembles.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGB', XGBClassifier())])))
results = []
names = []
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
for name, model in ensembles:
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
plt.figure()
model = Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesClassifier())])
train_sizes, train_scores, test_scores = learning_curve(model, X, y5_b, cv=kfold, n_jobs=1, train_sizes=np.linspace(.1,1.0,5))
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.grid()
plt.fill_between(train_sizes, train_scores_mean-train_scores_std, train_scores_mean+train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean-test_scores_std, test_scores_mean+test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training Score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-Validation Score")
plt.legend(loc="best")
plt.show()

In [ ]:
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_validation)
    
et = ExtraTreesClassifier()

# selecting features
sbs = SBS(et, k_features=5)
sbs.fit(X_train_std, Y_train)

# plotting performance of feature subsets
k_feat = [len(k) for k in sbs.subsets_]

plt.plot(k_feat, sbs.scores_, marker='o')
plt.ylim([0.7, 1.0])
plt.ylabel('Accuracy')
plt.xlabel('Number of features')
plt.grid()
plt.tight_layout()
# plt.savefig('./sbs.png', dpi=300)
plt.show()

In [ ]:
k5 = list(sbs.subsets_[10])
print(X.columns[1:][k5])
#print(k5)

In [ ]:
et.fit(X_train_std, Y_train)
print('Training accuracy:', et.score(X_train_std, Y_train))
print('Test accuracy:', et.score(X_test_std, Y_validation))

In [ ]:
et.fit(X_train_std[:, k5], Y_train)
print('Training accuracy:', et.score(X_train_std[:, k5], Y_train))
print('Test accuracy:', et.score(X_test_std[:, k5], Y_validation))